In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import json
from PIL import Image
import pandas as pd

# 범퍼 데이터 갯수 안맞음

In [8]:
# bumper_path = pd.read_csv("./data/data/source_data/bumper/bumper_path.csv")
# bumper_label = pd.read_csv("./data/data/label_data/bumper/bumper_label.csv")

# print(f"{bumper_path},{bumper_label}")

### 범퍼 데이터 병합 후 개수 조절

In [9]:
# bumper_data = pd.read_csv("./data/data/output/bumper/bumper_data.csv")
# print(f"Number of rows in CSV: {len(bumper_data)}")
# # print(df.duplicated().sum())  # 중복 행 개수 확인

# bumper_data = bumper_data.drop_duplicates()
# bumper_data.to_csv("./data/data/output/bumper/bumper_data.csv", index=False)


# 도어 데이터 갯수 안맞음

In [10]:
# door_path = pd.read_csv("./data/data/source_data/door/door_scratch_path.csv")
# door_label = pd.read_csv("./data/data/label_data/door/door_scratch_label.csv")

# print(f"{door_path},{door_label}")

## 도어 데이터 병합 후 개수 조절

In [11]:
door_data = pd.read_csv("./data/data/output/door/door_scratch_data.csv")
print(f"Number of rows in CSV: {len(door_data)}")
# print(df.duplicated().sum())  # 중복 행 개수 확인

# door_data = door_data.drop_duplicates()
# door_data.to_csv(
#     "./data/data/output/door/door_scratch_data.csv", index=False)

# print(door_data.iloc[:, 0])
# print(door_data.iloc[:, 1])

Number of rows in CSV: 1060


In [12]:
paths = door_data.iloc[:, 0].tolist()  # 첫 번째 열: 이미지 경로
labels = door_data.iloc[:, 1].tolist()  

In [13]:
print(len(paths))
print(len(labels))

1060
1060


In [14]:
# # 모델 정의
# class SimpleCNN(nn.Module):
#     def __init__(self):
#         super(SimpleCNN, self).__init__()
#         # conv 2번 / pooling 2번 / ReLU 1번
#         self.conv_Layers = nn.Sequential(
#             # 32 - 5 + 1 -> 28 => 32 * 32를 28 * 28 * 6 사이즈로 변환
#             nn.Conv2d(3, 6, 5),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2),  # stride -> 28 * 28 => 14 * 14
#             nn.Conv2d(6, 16, 5),  # 14 - 5 + 1 = 10 -> 14 * 14 => 10 * 10
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2)  # stride -> 10 * 10 => 5 * 5
#         )
#         self.flatten = nn.Flatten()
#         self.fc_layer = nn.Sequential(
#             nn.Linear(5*5*16, 120),
#             nn.ReLU(),
#             nn.Linear(120, 84),
#             nn.ReLU(),
#             nn.Linear(84, 10)
#         )  # fully connected layer

#     def forward(self, x):
#         out = self.conv_Layers(x)
#         flatten = self.flatten(out)
#         fc_out = self.fc_layer(flatten)
#         return fc_out


# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = SimpleCNN()
# model.to(device)
# print(device)

In [15]:
import torch.nn as nn
import torch.nn.functional as F


class ResNetLikeCNN(nn.Module):
    def __init__(self):
        super(ResNetLikeCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # 3x3 Conv
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, 10)  # Output layer (10 classes)

        self.pool = nn.MaxPool2d(2, 2)
        self.batchnorm1 = nn.BatchNorm2d(16)  # Batch Normalization
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.batchnorm3 = nn.BatchNorm2d(64)

    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(-1, 64 * 4 * 4)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNetLikeCNN()
model.to(device)

ResNetLikeCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batchnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [16]:
# 손실함수 정의
loss_fn = nn.CrossEntropyLoss()
# # 가중치 학습1
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# 가중치 학습2
# optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
# 가중치 학습3
# optimizer = optim.SGD(model.parameters(), lr=0.01,
#                       momentum=0.9, weight_decay=5e-4)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
# 가중치 학습 4
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

In [17]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import torch

# 데이터셋 클래스 정의 (커스텀 데이터셋)


class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


# 데이터 전처리 (Normalization 및 그레이스케일 변환)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # 그레이스케일로 변환 후 3채널로 확장
    transforms.RandomHorizontalFlip(),  # 데이터 증강 : 이미지 좌우 반전
    transforms.RandomCrop(32, padding=4),  # 데이터 증강 : padding 4 후 32*32로 랜덤 크롭
    transforms.ToTensor(),  # 텐서로 변환
    transforms.RandomRotation(10),  # 10도 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 밝기 및 대비 조정

    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])



# CustomDataset으로 데이터셋 만들기
train_dataset = CustomDataset(
    image_paths=paths, labels=labels, transform=transform)
# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# test_dataset = CustomDataset(
#     image_paths=source_test_files, labels=test_json_dataset, transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
from PIL import Image, ImageFile, UnidentifiedImageError
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 손상된 이미지를 처리하도록 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

# 손상된 이미지가 있으면 건너뛰기
def safe_load_image(image_path):
    try:
        image = Image.open(image_path)
        image.load()  # 이미지가 제대로 로드되는지 확인
        return image
    except (OSError, UnidentifiedImageError, ValueError):
        print(f"Skipping corrupted image: {image_path}")
        return None  # 손상된 이미지는 None 반환

# 데이터셋 클래스 정의 (커스텀 데이터셋)
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.transform = transform
        self.data = []

        for img_path, label in zip(image_paths, labels):
            # 이미지 로드 및 유효성 확인
            image = safe_load_image(img_path)
            if image is not None:
                self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]

        # 이미지를 다시 안전하게 로드
        image = safe_load_image(image_path)
        if self.transform:
            image = self.transform(image)

        return image, label

# 모델 학습
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):  # train_loader에서 가져오기
        # 손상된 이미지가 반환된 경우 건너뛰기
        if inputs is None:
            continue
        
        # 입력 데이터와 레이블을 device로 이동
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)  # 모델에 입력
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 학습률 스케줄러 업데이트
    # scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

print("학습 완료")



c:\Users\dawoo\Desktop\SF7\coding\PYTORCH\pytorch-env\Lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1, Loss: 0.7903274139937233
Epoch 2, Loss: 0.7014334605020636
Epoch 3, Loss: 0.6918510426493252
Epoch 4, Loss: 0.6976956251789542


In [ ]:
# # 테스트하기
# correct = 0
# total = len(test_loader.dataset)
# with torch.no_grad():  # test는 기울기 계산 X
#     for (images, labels) in test_loader:
#         # 손상된 이미지가 반환된 경우 건너뛰기
#         if images is None:
#             continue
        
#         images, labels = images.to(device), labels.to(device)  # 이미지를 GPU로 이동
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         correct += (predicted == labels).sum().item()

# print(f"Accuracy: {100 * correct / total}%")
